In [31]:
import torch
import json
import scipy.io as io

In [ ]:
net = io.loadmat('/mnt/hdd02/crownd_counting_results/mat_bagnet33_baseres18_net.mat')
print(net)

In [ ]:
params = io.loadmat('/mnt/hdd02/crownd_counting_results/bagnet33_params.mat')
for i, _ in enumerate(params['params'][0]):  
    print(params['params'][0][i][0], params['params'][0][i][1].shape)


In [ ]:
layers = io.loadmat('/mnt/hdd02/crownd_counting_results/bagnet33_layers.mat')
layers.keys()
layers['layers'][0][5]

In [ ]:
from models import bagnet33_res18
from torchsummary import summary

model = bagnet33_res18.bagnet33().cuda()
print(model)

In [ ]:
import torch.nn as nn

for i, layer in enumerate(model.modules()):
    print(i)
    if isinstance(layer, nn.Conv2d):
        print(layer.weight)
        layer.weight = torch.from_numpy(params['params'][0][i][1])
    elif isinstance(layer, nn.BatchNorm2d):
        layer.weight = torch.from_numpy(params['params'][0][i][1])
        layer.bias = torch.from_numpy(params['params'][0][i+1][1])
        i += 1

In [ ]:
from torchvision import models

model = models.resnet18(pretrained=True)
for p in model.bn1.parameters():
    print(p)

In [72]:
from my_transform import Gaussian_Filtering, Scale, Corner_Center_Crop, Random_Crop, Target_Scale, BagNet_Target_Scale
from datasets.ShanghaiTech_B import ShanghaiTech_B, target_loader, gt_mapping
import numpy as np
from PIL import Image
from 

crop = Random_Crop(448, 448)
gf = Gaussian_Filtering(15)
tar_S = Target_Scale(3)

In [55]:
import cv2

pic = cv2.imread('/mnt/hdd02/ShanghaiTech/part_B/train_data/images/IMG_1.jpg')
empty = np.zeros(pic.shape)
gt = io.loadmat('/mnt/hdd02/ShanghaiTech/part_B/train_data/ground_truth/GT_IMG_1.mat')

In [56]:
gt = gt['image_info'][0,0][0,0][0]

In [57]:
for location in gt:
    for i,xy in enumerate(location):
        if i == 0:
            x = int(xy)
        else:
            y = int(xy)
    out = cv2.circle(empty, (x, y), 5, (0, 0, 255), -1)

In [52]:
out.shape

(768, 1024, 3)

In [58]:
cv2.imwrite('empty.png', empty)

True

In [73]:
img = Image.open('/mnt/hdd02/ShanghaiTech/part_B/train_data/images/IMG_1.jpg')
target = gt_mapping(img, target_loader('/mnt/hdd02/ShanghaiTech/part_B/train_data/images/IMG_1.jpg'))

In [81]:
print(target.shape)
g_target = gf(target)

(768, 1024)


In [83]:
g_target.save('gaussian.png')

OSError: cannot write mode F as PNG